In [0]:

-- Lista de features para incluir
-- [X] QtdProdutos (ID) únicos (tamanho portfólio)
-- [X] Número de categorias distintas vendidas pelo seller
-- [X] Média da quantidade de fotos
-- [X] taxa de produtos com fotos (%)
-- [X] Peso médio dos produtos vendidos
-- [X] Média do tamanho do nome do produto
-- [X] Média do tamanho do nome da descrição
-- [X] Indicador de se existe descrição atrelada ao anúncio (%)
-- [X] Cubagem média dos produtos vendidos (H x L x C)
-- [ ] Produto mais vendido
-- [ ] Produto mais rentável
-- [ ] CategoriaPrincipal (mais vendida)
-- [ ] CategoriaPrincipal (maior receita)
-- [ ] Share receita nas top 15 categorias
-- [ ] Share qtde nas top 15 categorias


-- Criar a tabela base com todos os atributos necessários
WITH tb_base AS (
  SELECT v.idVendedor, 
         ip.idPedido, 
         p.dtPedido,
         ip.idProduto,
         ip.vlPreco AS precoProduto,
         ip.idPedidoItem,
         po.descCategoria,
         po.nrTamanhoNome,
         po.nrTamanhoDescricao,
         po.nrFotos,
         po.vlPesoGramas,
         po.vlComprimentoCm,
         po.vlAlturaCm,
         po.vlLarguraCm

  FROM silver.olist.vendedor AS v
  LEFT JOIN silver.olist.item_pedido AS ip ON v.idVendedor = ip.idVendedor
  LEFT JOIN silver.olist.pedido AS p ON ip.idPedido = p.idPedido
  LEFT JOIN silver.olist.produto AS po ON ip.idProduto = po.idProduto
  WHERE p.dtPedido < '2017-06-01'
),

-- Incluir os atributos da feature store na tabela
tb_feat_produto AS (
  SELECT idVendedor,
         COUNT(DISTINCT idProduto) AS qtdProdutosDistintos,
         COUNT(DISTINCT descCategoria) AS qtdCategoriasDistintas,
         AVG(nrFotos) AS mediaQtdeFotos,
         AVG(vlPesoGramas) AS mediaPesoGramas,
         COUNT(DISTINCT CASE WHEN nrFotos > 0 THEN idProduto END) / COUNT(DISTINCT idProduto) AS pctProdutosComFotos,
         AVG(nrTamanhoNome) AS mediaTamanhoNomeProduto,
         AVG(nrTamanhoDescricao) AS mediaTamanhoDescricaoProduto,
         AVG(vlComprimentoCm * vlAlturaCm * vlLarguraCm) AS mediaCubagemProduto,
         COUNT(DISTINCT CASE WHEN descCategoria IS NOT NULL THEN idProduto END) / COUNT(DISTINCT idProduto) AS pctProdutosComDescricao

  FROM tb_base
  GROUP BY idVendedor
  ORDER BY pctProdutosComFotos DESC
)

SELECT '2017-06-01' AS dtRef,
       *
FROM tb_feat_produto